In [1]:
import pylhe
import ROOT
import math
import os
import uproot
import numpy as np
import h5py
import time
t1 = time.time()

Welcome to JupyROOT 6.20/04


In [2]:
jjBG_rootpath = "/data/Chen-Wang/jjBG_200PT_1M/Events/run_01/tag_1_delphes_events.root"
#jjBG_rootpath = "/data/Chen-Wang/jjBG_200PT/Events/run_01/tag_1_delphes_events.root"
jjBG_file = uproot.open(jjBG_rootpath)

In [3]:
##### include jjBG data #####
jjBG_events = [jjBG_file["Delphes;1"]["Particle.Status"].array(),
        jjBG_file["Delphes;1"]["Particle.PT"].array(),
        jjBG_file["Delphes;1"]["Particle.Eta"].array(),
        jjBG_file["Delphes;1"]["Particle.Phi"].array(),
        jjBG_file["Delphes;1"]["FatJet.Particles"].array(),
        jjBG_file["Delphes;1"]["FatJet.Eta"].array(),
        jjBG_file["Delphes;1"]["FatJet.Phi"].array(),
        jjBG_file["Delphes;1"]["FatJet.PT"].array(),
        jjBG_file["Delphes;1"]["FatJet"].array(),
        jjBG_file["Delphes;1"]["FatJet.Mass"].array()
        ]

##### reshape the jjBG data #####

jjBG_events = np.expand_dims(jjBG_events, axis=-1)
jjBG_events = jjBG_events.transpose((1,0,2))
jjBG_events = np.squeeze(jjBG_events,axis=(2,))

num_jjBG = len(jjBG_events)

t2 = time.time()
print("Time:", t2-t1)

Time: 305.3268744945526


In [4]:
#####   useful function    #####
##### select if Fat Jet>=2 #####

def Fat_Jet_selection(events):
    where1 = np.where(events[:,8]>=2)
    return events[where1]

##### select if M_jet > 50 GeV #####

def mass_selection(events):
    where1 = []
    for i in range(len(events)):
        switch=1
        if events[i][9][0]<50:
            switch=0
        if events[i][9][1]<50:
            switch=0
        if switch==1:
            where1.append(i)
    return events[where1]
##### define figure generator #####

def PT_generator(event):
    grid_num = 50    # means n^2 points
    where_jet_particle = event[4][0]+event[4][1]
    where_jet_particle = np.array(where_jet_particle)-1
    particle_Eta = event[2][where_jet_particle] - event[5][0]
    particle_Phi = event[3][where_jet_particle] - event[6][0]
    particle_PT = event[1][where_jet_particle]
    
    xedges = np.linspace(-5, 5, grid_num+1)        # Eta limit
    yedges = np.linspace(-5, 5, grid_num+1)    # Phi limit
    PT = np.zeros([grid_num,grid_num])
    
    for Eta in range(grid_num):
        for Phi in range(grid_num):
            where1 = np.where((xedges[Eta]<particle_Eta) & (particle_Eta<xedges[Eta+1]) & (yedges[Phi]<particle_Phi) & (particle_Phi<yedges[Phi+1]))
            PT[Eta][Phi] = sum(particle_PT[where1])
            
    return PT.T    ### after transpose: X=eta Y=phi

##### define leading jet generator #####

def leadingJ_generator(jet_events):
    grid_num = 50    # means n^2 points
    where_jet_particle = jet_events[4][0]
    where_jet_particle = np.array(where_jet_particle)-1
    particle_Eta = jet_events[2][where_jet_particle] - jet_events[5][0]    # do the centerize
    particle_Phi = jet_events[3][where_jet_particle] - jet_events[6][0]    # do the centerize
    particle_PT = jet_events[1][where_jet_particle]
    
    xedges = np.linspace(-1, 1, grid_num+1)    # Eta limit
    yedges = np.linspace(-1, 1, grid_num+1)    # Phi limit
    PT = np.zeros([grid_num,grid_num])
    
    for Eta in range(grid_num):
        for Phi in range(grid_num):
            where1 = np.where((xedges[Eta]<particle_Eta) & (particle_Eta<xedges[Eta+1]) & (yedges[Phi]<particle_Phi) & (particle_Phi<yedges[Phi+1]))
            PT[Eta][Phi] = sum(particle_PT[where1])
    
    leading_b = np.where(PT==np.max(PT))
    PT[leading_b[0][0]-4:leading_b[0][0]+4, leading_b[1][0]-4:leading_b[1][0]+4] = 0
    subleading_b = np.where(PT==np.max(PT))

    eta_diff = (subleading_b[0][0]-leading_b[0][0])*(1/grid_num)     # each grid different (1/grid_num)
    phi_diff = (subleading_b[1][0]-leading_b[1][0])*(1/grid_num)     # each grid different (1/grid_num)
    cos_theta = (0*eta_diff + -1*phi_diff)/(np.sqrt(eta_diff**2 + phi_diff**2)+10**(-5))     # inner product with (0,-1)
    sin_theta = np.sqrt(1-cos_theta**2) if eta_diff>0 else -np.sqrt(1-cos_theta**2)

    particle_Eta_rot = cos_theta*particle_Eta + sin_theta*particle_Phi
    particle_Phi_rot = -sin_theta*particle_Eta + cos_theta*particle_Phi

    for Eta_rot in range(grid_num):
        for Phi_rot in range(grid_num):
            where1 = np.where((xedges[Eta_rot]<particle_Eta_rot) & (particle_Eta_rot<xedges[Eta_rot+1]) & (yedges[Phi_rot]<particle_Phi_rot) & (particle_Phi_rot<yedges[Phi_rot+1]))
            PT[Eta_rot][Phi_rot] = sum(particle_PT[where1])
    
    return PT.T

##### define sub-leading jet generator #####

def subleadingJ_generator(jet_events):
    grid_num = 50    # means n^2 points
    where_jet_particle = jet_events[4][1]
    where_jet_particle = np.array(where_jet_particle)-1
    particle_Eta = jet_events[2][where_jet_particle] - jet_events[5][1]    # do the centerize
    particle_Phi = jet_events[3][where_jet_particle] - jet_events[6][1]    # do the centerize
    particle_PT = jet_events[1][where_jet_particle]
    
    xedges = np.linspace(-1, 1, grid_num+1)    # Eta limit
    yedges = np.linspace(-1, 1, grid_num+1)    # Phi limit
    PT = np.zeros([grid_num,grid_num])
    
    for Eta in range(grid_num):
        for Phi in range(grid_num):
            where1 = np.where((xedges[Eta]<particle_Eta) & (particle_Eta<xedges[Eta+1]) & (yedges[Phi]<particle_Phi) & (particle_Phi<yedges[Phi+1]))
            PT[Eta][Phi] = sum(particle_PT[where1])
    
    leading_b = np.where(PT==np.max(PT))
    PT[leading_b[0][0]-4:leading_b[0][0]+4, leading_b[1][0]-4:leading_b[1][0]+4] = 0
    subleading_b = np.where(PT==np.max(PT))

    eta_diff = (subleading_b[0][0]-leading_b[0][0])*(1/grid_num)     # each grid different (1/grid_num)
    phi_diff = (subleading_b[1][0]-leading_b[1][0])*(1/grid_num)     # each grid different (1/grid_num)
    cos_theta = (0*eta_diff + -1*phi_diff)/(np.sqrt(eta_diff**2 + phi_diff**2)+10**(-5))     # inner product with (0,-1)
    sin_theta = np.sqrt(1-cos_theta**2) if eta_diff>0 else -np.sqrt(1-cos_theta**2)

    particle_Eta_rot = cos_theta*particle_Eta + sin_theta*particle_Phi
    particle_Phi_rot = -sin_theta*particle_Eta + cos_theta*particle_Phi

    for Eta_rot in range(grid_num):
        for Phi_rot in range(grid_num):
            where1 = np.where((xedges[Eta_rot]<particle_Eta_rot) & (particle_Eta_rot<xedges[Eta_rot+1]) & (yedges[Phi_rot]<particle_Phi_rot) & (particle_Phi_rot<yedges[Phi_rot+1]))
            PT[Eta_rot][Phi_rot] = sum(particle_PT[where1])
    
    return PT.T

In [5]:
##### select only fat jet >=2 #####

jjBG_events = Fat_Jet_selection(jjBG_events)
jjBG_events = mass_selection(jjBG_events)
print("There are", len(jjBG_events), "jjBG events survive with originally ", num_jjBG, " events.")

t2 = time.time()
print("Time:", t2-t1)

There are 270777 jjBG events survive with originally  1000000  events.
Time: 312.7449872493744


In [6]:
jjBG = []
for events in range(len(jjBG_events)):
    if (events%50000==0):
        print("check point, time:", time.time()-t1)
    jjBG.append([PT_generator(jjBG_events[events]), leadingJ_generator(jjBG_events[events]), subleadingJ_generator(jjBG_events[events])])
    
t2 = time.time()
print("Time:", t2-t1)

check point, time: 312.8037054538727
check point, time: 7439.008726358414
check point, time: 14569.864250659943
check point, time: 21702.75822854042
check point, time: 28840.40584397316
check point, time: 35981.77191543579
Time: 38949.622113227844


In [7]:
h5f = h5py.File('/data/Chen-Wang/data_image.h5', 'a')
h5f.create_dataset('jjBG', data=jjBG)
h5f.close()

In [8]:
t2 = time.time()
print("total time:", t2-t1)

total time: 38968.281178474426
